## Praca na zajęciach 25.03.24

### Kateryna Kolioglo

1. Napisz program z jedną zmienną **"globalną"**, która jest całkowitą i zaczyna od wartości niezerowej.
Program powinien mieć dwa wątki:
- `jeden ma zwiększać daną globalną o jeden`
- `drugi zmniejszać o 1`

oba co sekundę.
Po 10 sekundach należy wypisać wartość zmiennej na ekran i zakończyć działanie programu, gwarantując poprawne zatrzymanie wątków.

In [5]:
public class ex1 {
    private static int globalVariable = 1; // Zmienna globalna

    public static void main(String[] args) {
        Thread incrementThread = new Thread(new IncrementTask());
        Thread decrementThread = new Thread(new DecrementTask());

        incrementThread.start();
        decrementThread.start();

        try {
            Thread.sleep(10000); // Czekaj 10 sekund
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        incrementThread.interrupt(); // Przerwij wątek inkrementacji
        decrementThread.interrupt(); // Przerwij wątek dekrementacji

        // Poczekaj na zakończenie obu wątków
        try {
            incrementThread.join();
            decrementThread.join();
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        // Wyświetl wartość zmiennej globalnej
        System.out.println("Wartość zmiennej globalnej po 10 sekundach: " + globalVariable);
    }

    static class IncrementTask implements Runnable {
        @Override
        public void run() {
            while (!Thread.currentThread().isInterrupted()) {
                globalVariable++;
                try {
                    Thread.sleep(1000);
                } catch (InterruptedException e) {
                    Thread.currentThread().interrupt();
                }
            }
        }
    }

    static class DecrementTask implements Runnable {
        @Override
        public void run() {
            while (!Thread.currentThread().isInterrupted()) {
                globalVariable--;
                try {
                    Thread.sleep(1000);
                } catch (InterruptedException e) {
                    Thread.currentThread().interrupt();
                }
            }
        }
    }
}

In [6]:
ex1.main(null)

Wartość zmiennej globalnej po 10 sekundach: 0


Albo można inaczej, przy użyciu klasy java.util.concurrent.atomic.AtomicInteger, która zapewnia atomowe operacje na liczbach całkowitych bez potrzeby jawnej synchronizacji:

In [9]:
import java.util.concurrent.atomic.AtomicInteger;

public class AtomicExample {
    private static AtomicInteger globalVariable = new AtomicInteger(1); // Atomic zmienna

    public static void main(String[] args) {
        Thread incrementThread = new Thread(new IncrementTask());
        Thread decrementThread = new Thread(new DecrementTask());

        incrementThread.start();
        decrementThread.start();

        try {
            Thread.sleep(10000); // Oczekiwanie 10 sekund 
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        incrementThread.interrupt(); // Przerwanie przepływu inkrementacji
        decrementThread.interrupt(); // Przerwanie przepływu dekrementacji

        // Oczekiwanie na zakończenie obu wątków
        try {
            incrementThread.join();
            decrementThread.join();
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        // Wyświetla wartość zmiennej globalnej
        System.out.println("Wartość zmiennej globalnej po 10 sekundach: " + globalVariable);
    }

    static class IncrementTask implements Runnable {
        @Override
        public void run() {
            while (!Thread.currentThread().isInterrupted()) {
                globalVariable.getAndIncrement();
                try {
                    Thread.sleep(1000);
                } catch (InterruptedException e) {
                    Thread.currentThread().interrupt();
                }
            }
        }
    }

    static class DecrementTask implements Runnable {
        @Override
        public void run() {
            while (!Thread.currentThread().isInterrupted()) {
                globalVariable.getAndDecrement();
                try {
                    Thread.sleep(1000);
                } catch (InterruptedException e) {
                    Thread.currentThread().interrupt();
                }
            }
        }
    }
}


In [11]:
AtomicExample.main(null)

Wartość zmiennej globalnej po 10 sekundach: 1


2. Zaimplementuj metody:
* `public static void runTogether(Runnable... tasks)`
* `public static void runInOrder(Runnable... tasks)`
  
Pierwsza metoda powinna uruchomić każde zadanie w oddzielnym wątku i zakończyć działanie. Druga metoda powinna uruchomić wszystkie zadania w bieżącym wątku i zakończył działanie po zakończeniu ostatniego z nich.

In [14]:
public class ex2 {
    public static void main(String[] args) {
        // Przykładowe użycie metody runTogether
        Runnable task1 = () -> System.out.println("Zadanie 1 wykonane w wątku: " + Thread.currentThread().getName());
        Runnable task2 = () -> System.out.println("Zadanie 2 wykonane w wątku: " + Thread.currentThread().getName());
        Runnable task3 = () -> System.out.println("Zadanie 3 wykonane w wątku: " + Thread.currentThread().getName());

        runTogether(task1, task2, task3);

        // Przykładowe użycie metody runInOrder
        Runnable task4 = () -> System.out.println("Zadanie 4 wykonane w wątku: " + Thread.currentThread().getName());
        Runnable task5 = () -> System.out.println("Zadanie 5 wykonane w wątku: " + Thread.currentThread().getName());
        Runnable task6 = () -> System.out.println("Zadanie 6 wykonane w wątku: " + Thread.currentThread().getName());

        runInOrder(task4, task5, task6);
    }

    public static void runTogether(Runnable... tasks) {
        for (Runnable task : tasks) {
            Thread thread = new Thread(task);
            thread.start();
        }
    }

    public static void runInOrder(Runnable... tasks) {
        for (Runnable task : tasks) {
            task.run();
        }
    }
}

In [15]:
ex2.main(null)

Zadanie 2 wykonane w wątku: Thread-14
Zadanie 1 wykonane w wątku: Thread-13
Zadanie 3 wykonane w wątku: Thread-15
Zadanie 4 wykonane w wątku: IJava-executor-4
Zadanie 5 wykonane w wątku: IJava-executor-4
Zadanie 6 wykonane w wątku: IJava-executor-4


3. Korzystając z metod `listFiles(FileFilter)` i `isDirectory` z klasy `java.io.File`, napisz metodę zwracającą wszystkie podkatalogi wskazanego katalogu. Wykorzystaj wyrażenie `lambda` zamiast obiektu `FileFilter`. Wykonaj to samo za pomocą wyrażenia z metodą i `anonimowej` klasy wewnętrznej.

In [18]:
import java.io.File;
import java.io.FileFilter;

public class ex3 {
    public static void main(String[] args) {
        // Wyrażenie lambda
        File directory = new File("AJD");
        File[] subDirectoriesLambda = getSubDirectories(directory, file -> file.isDirectory());
        if (subDirectoriesLambda != null) {
            System.out.println("Podkatalogi (lambda):");
            printSubDirectories(subDirectoriesLambda);
        } else {
            System.out.println("Brak podkatalogów (lambda)");
        }

        // Anonimowa klasa wewnętrzna
        File[] subDirectoriesAnonymous = getSubDirectories(directory, new FileFilter() {
            @Override
            public boolean accept(File file) {
                return file.isDirectory();
            }
        });
        if (subDirectoriesAnonymous != null) {
            System.out.println("\nPodkatalogi (anonimowa klasa wewnętrzna):");
            printSubDirectories(subDirectoriesAnonymous);
        } else {
            System.out.println("Brak podkatalogów (anonimowa klasa wewnętrzna)");
        }
    }

    public static File[] getSubDirectories(File directory, FileFilter filter) {
        if (directory.exists() && directory.isDirectory()) {
            return directory.listFiles(filter);
        } else {
            System.out.println("Podany plik nie istnieje lub nie jest katalogiem.");
            return null;
        }
    }

    public static void printSubDirectories(File[] subDirectories) {
        for (File subDirectory : subDirectories) {
            System.out.println(subDirectory.getName());
        }
    }
}

In [19]:
ex3.main(null)

Podany plik nie istnieje lub nie jest katalogiem.
Brak podkatalogów (lambda)
Podany plik nie istnieje lub nie jest katalogiem.
Brak podkatalogów (anonimowa klasa wewnętrzna)
